In [3]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output


In [7]:
net = LeNet()
print(net)


LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [8]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)


In [9]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进。
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n


In [10]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))


In [11]:
lr, num_epochs = 0.001, 4
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)


training on  cpu
epoch 1, loss 1.8984, train acc 0.295, test acc 0.585, time 17.9 sec
epoch 2, loss 0.9640, train acc 0.632, test acc 0.671, time 18.7 sec
epoch 3, loss 0.8044, train acc 0.701, test acc 0.723, time 17.6 sec
epoch 4, loss 0.7126, train acc 0.733, test acc 0.737, time 17.8 sec


In [7]:
# 用来看每一层的参数数值
for parameters in net.parameters():
    print(parameters)



Parameter containing:
tensor([[[[ 0.5685,  0.6889,  0.5448,  0.3617,  0.0330],
          [ 0.5997,  0.5978,  0.6705,  0.3932,  0.2447],
          [ 0.3531,  0.6739,  0.6318,  0.4803, -0.1409],
          [ 0.0651,  0.7484,  0.4966,  0.1406, -0.1883],
          [ 0.0295,  0.6772,  0.6648,  0.4001, -0.0748]]],


        [[[-0.2262,  0.3684,  0.5605,  0.3941, -0.0595],
          [-0.5119,  0.2537,  0.3053,  0.3877, -0.1327],
          [-0.4908,  0.2168,  0.1972,  0.4907, -0.0435],
          [-0.2976,  0.1258,  0.3913,  0.3393, -0.0495],
          [-0.4859, -0.0028,  0.2366,  0.3441, -0.0508]]],


        [[[ 0.0209,  0.5836,  0.8003,  0.4966,  0.4398],
          [ 0.0651,  0.3117,  0.8224,  0.5957,  0.4354],
          [ 0.1330,  0.1816,  0.7927,  0.4619,  0.4352],
          [-0.2228,  0.1511,  0.7073,  0.3523,  0.3850],
          [-0.1821,  0.3215,  0.5523,  0.5285,  0.3123]]],


        [[[-0.1340, -0.1844, -0.2721, -0.0612,  0.0205],
          [-0.1622, -0.3484, -0.6399, -0.6211,  0.0551

In [14]:
# 用来看整体结构
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())



conv.0.weight : torch.Size([6, 1, 5, 5])
conv.0.bias : torch.Size([6])
conv.3.weight : torch.Size([16, 6, 5, 5])
conv.3.bias : torch.Size([16])
fc.0.weight : torch.Size([120, 256])
fc.0.bias : torch.Size([120])
fc.2.weight : torch.Size([84, 120])
fc.2.bias : torch.Size([84])
fc.4.weight : torch.Size([10, 84])
fc.4.bias : torch.Size([10])


In [13]:
## 保存模型

model = net
torch.save(model.state_dict(), "/Users/mac/model_LeNet.pth")


In [4]:
## 导入模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output
net = LeNet()
model = net
model.load_state_dict(torch.load("/Users/mac/model_LeNet.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [3]:
import matplotlib.pyplot as plt
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
import numpy as np
from torchvision.datasets import ImageFolder


In [47]:
simple_transform = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.ToTensor(),  # H, W, C -> C, W, H 归一化到(0,1)，简单直接除以255
                                       transforms.Normalize([0.485, 0.456, 0.406],  # std
                                                            [0.229, 0.224, 0.225])])


In [49]:
class LayerActivations:
    features = None
 
    def __init__(self, model, layer_num):
        self.hook = model[layer_num].register_forward_hook(self.hook_fn)
 
    def hook_fn(self, module, input, output):
        self.features = output.cpu()
 
    def remove(self):
        self.hook.remove()


In [61]:
print(net.conv)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): Sigmoid()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [60]:
train = ImageFolder(os.path.join("/Users", "mac","Desktop","test"), simple_transform)
valid = ImageFolder(os.path.join("/Users", "mac","Desktop","test"), simple_transform)
train_loader = DataLoader(train, batch_size=1,  shuffle=False, num_workers=5)
val_loader = DataLoader(valid, batch_size=1, shuffle=False, num_workers=5)

model = net.conv

In [84]:
conv_out = LayerActivations(model, 0)
img = next(iter(train_loader))[0]
act = conv_out.features

In [89]:
### 方法失败......

In [33]:
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import models

In [14]:
from PIL import Image
img = Image.open('/Users/mac/Desktop/test2.jpg')
if img.mode != 'L':
    img = img.convert('L')
out = img
out.save("/Users/mac/Desktop/test2.jpg")


In [107]:
## 这个代码是用来看黑白图片的，如果要看RGB，需要：改mean、std；不提取某一层；只进行一次加层
def preprocess_image(cv2im, resize_im=True):
    # mean and std
    ##mean = [0.485, 0.456, 0.406]
    ##std = [0.229, 0.224, 0.225]
    mean = [0.456]
    std = [0.225]
    # Resize
    if resize_im:
        cv2im = cv2.resize(cv2im, (224, 224))
    im_as_arr = np.float32(cv2im)
    im_as_arr = np.ascontiguousarray(im_as_arr[..., ::-1])
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    im_as_arr = im_as_arr[0,:,:]
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        ##im_as_arr[channel] -= mean[channel]
        ##im_as_arr[channel] /= std[channel]
        im_as_arr[channel] -= mean
        im_as_arr[channel] /= std
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel
    im_as_ten.unsqueeze_(0)
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var


class FeatureVisualization():
    def __init__(self,img_path,selected_layer):
        self.img_path=img_path
        self.selected_layer=selected_layer
        self.pretrained_model = net.conv

    def process_image(self):
        img=cv2.imread(self.img_path)
        img=preprocess_image(img)
        return img

    def get_feature(self):
        input=self.process_image()
        print(input.shape)
        x=input
        for index,layer in enumerate(self.pretrained_model):
            x=layer(x)
            if (index == self.selected_layer):
                return x

    def get_single_feature(self):
        features=self.get_feature()
        print(features.shape)

        ## 改变这里的数字，看同一层不同序号的图片
        feature=features[:,0,:,:]
        print(feature.shape)

        feature=feature.view(feature.shape[1],feature.shape[2])
        print(feature.shape)

        return feature

    def save_feature_to_img(self):
        #to numpy
        feature=self.get_single_feature()
        feature=feature.data.numpy()

        #use sigmod to [0,1]
        feature= 1.0/(1+np.exp(-1*feature))

        # to [0,255]
        feature=np.round(feature*255)
        print(feature[0])

        cv2.imwrite('/Users/mac/Desktop/img.jpg',feature)


In [108]:
img=cv2.imread("/Users/mac/Desktop/test2.jpg")
img = cv2.resize(img, (224, 224))
im_as_arr = np.float32(img)
im_as_arr = im_as_arr.transpose(2, 0, 1)
im_as_arr = im_as_arr[0,:,:]
im_as_ten = torch.from_numpy(im_as_arr).float()
im_as_ten.unsqueeze_(0)
im_as_ten.unsqueeze_(0)
im_as_ten.shape
mean = [0.456]
std = [0.225]
for channel, _ in enumerate(im_as_arr):
    im_as_arr[channel] /= 255
    im_as_arr[channel] -= mean
    im_as_arr[channel] /= std

In [109]:
## 改变这里的号码，看不同层
myClass=FeatureVisualization("/Users/mac/Desktop/test2.jpg",1)

In [110]:
myClass.save_feature_to_img()

torch.Size([1, 1, 224, 224])
torch.Size([1, 6, 220, 220])
torch.Size([1, 220, 220])
torch.Size([220, 220])
[128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128. 128.
 128. 128. 128. 128. 128. 128. 128. 128. 